### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [20]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&fl={field_list}&sort={sort}&begin_date={begin_date}&end_date={end_date}"

In [21]:
# Create an empty list to store the reviews
reviews_list = []


# loop through pages 0-19
for page in range(0,20):
    # create query with a page number
    # API results show 10 articles at a time
    page_based_query = f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(page_based_query).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
    

        # Print the page that was just retrieved
        print("Page number: {} has been retrieved.".format(page))
    
        #The page number has no result
    except IndexError:
        # Print the page number that had no results then break from the loop
        print("Page number: {} has no results.".format(page))
        break



Page number: 0 has been retrieved.
Page number: 1 has been retrieved.
Page number: 2 has been retrieved.
Page number: 3 has been retrieved.
Page number: 4 has been retrieved.
Page number: 5 has been retrieved.
Page number: 6 has been retrieved.
Page number: 7 has been retrieved.
Page number: 8 has been retrieved.
Page number: 9 has been retrieved.
Page number: 10 has been retrieved.
Page number: 11 has been retrieved.
Page number: 12 has been retrieved.
Page number: 13 has been retrieved.
Page number: 14 has been retrieved.
Page number: 15 has been retrieved.
Page number: 16 has been retrieved.
Page number: 17 has been retrieved.
Page number: 18 has been retrieved.
Page number: 19 has been retrieved.


In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5],indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)

In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
reviews_df["title"] = reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early


In [8]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords


# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)

In [9]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df["title"].to_list()

['The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'Pamela, a Love Story', 'In From the Side', 'After Love', 'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley’s Lover', 'The Sound of Christmas', 'The Inspection', 'Bones and All', 'My Policeman', 'About Fate', 'Waiting for Bojangles', 'I Love My Dad', 'A Love Song', 'Alone Together', 'Art of Love', 'The Wheel', 'Thor: Love and Thunder', 'Both Sides of the Blade', 'Fire of Love', 'Love & Gelato', 'Stay Prayed Up', 'Benediction', 'Dinner in America', 'In a New York Minute', 'Anaïs in Love', 'I Love America', 'See You Then', 'La Mami', 'Love After Love', 'Deep Water', 'Lucy and Desi', 'Cyrano', 'The In Between', 'Book of Love', 'Lingui, the Sacred Bonds', 'The Pink Cloud', 'A Journal for Jordan', 'West Side Story'

### Access The Movie Database API

In [10]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [11]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter%50==0:
        print("Sleeping for 1 second")
        time.sleep(1)

    # Add 1 to the request counter
    request_counter +=1
    
    # Perform a "GET" request for The Movie Database
    movie_query = f"{url}{title}{tmdb_key_string}"
    movie_search_query_response = requests.get(movie_query).json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = movie_search_query_response["results"][0]["id"]

        # Make a request for a the full movie details
        movie_details_query = f"https://api.themoviedb.org/3/movie/{movie_id}?{tmdb_key_string}"
        
        # Execute "GET" request with url
        movie_details = requests.get(movie_details_query).json()
        
        # Extract the genre names into a list
        genres = [item["name"] for item in movie_details["genres"]]
        

        # Extract the spoken_languages' English name into a list
        spoken_languages = [item["english_name"] for item in movie_details["spoken_languages"]]


        # Extract the production_countries' name into a list
        production_countries = [item["name"] for item in movie_details["production_countries"]]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_full_details = {}
        movie_full_details["title"] = movie_details["title"]
        movie_full_details["original_title"] = movie_details["original_title"]
        movie_full_details["budget"] = movie_details["budget"]
        movie_full_details["original_language"] = movie_details["original_language"]
        movie_full_details["homepage"] = movie_details["homepage"]
        movie_full_details["overview"] = movie_details["overview"]
        movie_full_details["popularity"] =  movie_details["popularity"]
        movie_full_details["runtime"] = movie_details["runtime"]
        movie_full_details["revenue"] = movie_details["revenue"]
        movie_full_details["release_date"] = movie_details["release_date"]
        movie_full_details["vote_average"] = movie_details["vote_average"]
        movie_full_details["vote_count"] = movie_details["vote_count"]
        
        movie_full_details["genres"] = genres
        movie_full_details["spoken_languages"] = spoken_languages
        movie_full_details["production_countries"] = production_countries

        tmdb_movies_list.append(movie_full_details)
        # Print out the title that was found
        print("{} has been found.".format(movie_details["title"]))

    except:
        print("{} has not been found.".format(movie_details["title"]))




The Attachment Diaries has been found.
The Attachment Diaries has not been found.
You Can Live Forever has been found.
A Tourist's Guide to Love has been found.
Other People's Children has been found.
One True Loves has been found.
The Lost Weekend: A Love Story has been found.
A Thousand and One has been found.
Your Place or Mine has been found.
Love in the Time of Fentanyl has been found.
Pamela, A Love Story has been found.
In from the Side has been found.
After Love has been found.
Alcarràs has been found.
Nelly and Monsieur Arnaud has been found.
Lady Chatterley's Lover has been found.
The Sound of Christmas has been found.
The Inspection has been found.
Bones and All has been found.
My Policeman has been found.
About Fate has been found.
Waiting for Bojangles has been found.
I Love My Dad has been found.
A Love Song has been found.
Alone Together has been found.
The Art of Love has been found.
2099: The Soldier Protocol has been found.
Thor: Love and Thunder has been found.
Both 

In [12]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5],indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.101,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 4.0,
        "vote_count": 2,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

In [13]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

### Merge and Clean the Data for Export

In [14]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df,tmdb_df,on="title")

In [15]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genres","spoken_languages","production_countries"]

# Create a list of characters to remove
characters_to_remove = ["[","]","'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype("str")

    # Loop through characters to remove
    for character in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(character,"")


# Display the fixed DataFrame
print(merged_df.head())


                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/you-...   
2  https://www.nytimes.com/2023/04/13/movies/one-...   
3  https://www.nytimes.com/2023/04/13/movies/the-...   
4  https://www.nytimes.com/2023/03/30/movies/a-th...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Religion comes between two girls falling in lo...  The New York Times   
2  A film adaptation of Taylor Jenkins Reid’s nov...  The New York Times   
3  There’s not much Lennon music heard in this do...  The New York Times   
4  A mesmerizing Teyana Taylor stars in A.V. Rock...  The New York Times   

                                            keywords  \
0  subject: Movies;creative_works: The Attachment...   
1  subject: Movies;creative_works: You Can Live F...   
2  subject: Movies;persons: Bracey, Lu

In [16]:
# Drop "byline.person" column
merged_df.drop("byline.person",axis=1,inplace=True)


In [17]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(inplace=True,drop=True)

In [18]:
# Export data to CSV without the index
merged_df.to_csv("./output/final_data.csv",index=False)